# **FEMA Data Pipeline for ETL**
---

## *Phase 1*

### Dependencies

In [1]:
# Main ETL Libraries
import pandas as pd
import requests
from concurrent.futures import ThreadPoolExecutor
from itertools import cycle
from time import sleep

# Formating and Display Libraries
from pprint import pprint

# File Library
from pathlib import Path

# Personal API Key File (Please use your own or comment out)!
import api_keys

### Extraction

In [2]:
# Reading CSV File into Pandas
fema_decl = pd.read_csv('Assets/CSVs/DisasterDeclarationsSummaries.csv', low_memory = False)
fema_decl.head()

,femaDeclarationString,disasterNumber,state,declarationType,declarationDate,fyDeclared,incidentType,declarationTitle,ihProgramDeclared,iaProgramDeclared,...,placeCode,designatedArea,declarationRequestNumber,lastIAFilingDate,incidentId,region,designatedIncidentTypes,lastRefresh,hash,id
0,FM-5530-NV,5530,NV,FM,2024-08-12T00:00:00.000Z,2024,Fire,GOLD RANCH FIRE,0,0,...,99031,Washoe (County),24123,NaN,2024081201,9,R,2024-08-27T18:22:14.800Z,5d07e7c51bb300bfbec94a699a1e1ab1d61a97cd,f15a7a79-f1c3-41bb-8a5c-c05fbae34423
1,FM-5529-OR,5529,OR,FM,2024-08-09T00:00:00.000Z,2024,Fire,LEE FALLS FIRE,0,0,...,99067,Washington (County),24122,NaN,2024081001,10,R,2024-08-27T18:22:14.800Z,ae87cf3c6ed795015b714af7166c7c295b2b67c7,09e3f81a-5e16-4b72-b317-1c64e0cfa59c
2,FM-5528-OR,5528,OR,FM,2024-08-06T00:00:00.000Z,2024,Fire,ELK LANE FIRE,0,0,...,99031,Jefferson (County),24116,NaN,2024080701,10,R,2024-08-27T18:22:14.800Z,432cf0995c47e3895cea696ede5621b810460501,59983f89-30bf-4888-b21b-62e8d57d9aac
3,FM-5527-OR,5527,OR,FM,2024-08-02T00:00:00.000Z,2024,Fire,MILE MARKER 132 FIRE,0,0,...,99017,Deschutes (County),24111,NaN,2024080301,10,R,2024-08-27T18:22:14.800Z,2f21d90cb6bc64b0d4121aa3f18d852bbb4b11fa,8d13ecf0-bc2f-496b-8c9f-b2e73da832a0
4,FM-5522-CA,5522,CA,FM,2024-07-27T00:00:00.000Z,2024,Fire,BOREL FIRE,0,0,...,99029,Kern (County),24102,NaN,2024072701,9,R,2024-08-27T18:22:14.800Z,51ec819011c936f387edf10b9d5839b35419ca95,9b73e19b-d326-4992-8da1-7e658d97607c


### Basic Cleanup

#### Column Removal & DT Slicing

In [3]:
# Removing unwanted/unusable columns
fema_decl_clean = fema_decl[
    [
        'disasterNumber', 'declarationRequestNumber', 'declarationTitle', 'incidentType', 
        'declarationType', 'designatedArea', 'state', 'region', 'declarationDate', 
        'incidentBeginDate', 'tribalRequest', 'ihProgramDeclared', 'iaProgramDeclared', 
        'paProgramDeclared', 'hmProgramDeclared', 'id'
    ]
].copy()

# Slicing Dates to extract just the YYYY-MM-DD
fema_decl_clean['declarationDate'] = fema_decl_clean['declarationDate'].str.slice_replace(10)
fema_decl_clean['incidentBeginDate'] = fema_decl_clean['incidentBeginDate'].str.slice_replace(10)

display(fema_decl_clean)

,disasterNumber,declarationRequestNumber,declarationTitle,incidentType,declarationType,designatedArea,state,region,declarationDate,incidentBeginDate,tribalRequest,ihProgramDeclared,iaProgramDeclared,paProgramDeclared,hmProgramDeclared,id
0,5530,24123,GOLD RANCH FIRE,Fire,FM,Washoe (County),NV,9,2024-08-12,2024-08-11,0,0,0,1,1,f15a7a79-f1c3-41bb-8a5c-c05fbae34423
1,5529,24122,LEE FALLS FIRE,Fire,FM,Washington (County),OR,10,2024-08-09,2024-08-08,0,0,0,1,1,09e3f81a-5e16-4b72-b317-1c64e0cfa59c
2,5528,24116,ELK LANE FIRE,Fire,FM,Jefferson (County),OR,10,2024-08-06,2024-08-04,0,0,0,1,1,59983f89-30bf-4888-b21b-62e8d57d9aac
3,5527,24111,MILE MARKER 132 FIRE,Fire,FM,Deschutes (County),OR,10,2024-08-02,2024-08-02,0,0,0,1,1,8d13ecf0-bc2f-496b-8c9f-b2e73da832a0
4,5522,24102,BOREL FIRE,Fire,FM,Kern (County),CA,9,2024-07-27,2024-07-25,0,0,0,1,1,9b73e19b-d326-4992-8da1-7e658d97607c
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67351,9,53010,FLOOD,Flood,DR,Statewide,TX,6,1953-06-19,1953-06-19,0,0,1,1,1,6ef68fa6-5889-466b-9e3b-ee4c06da0876
67352,8,53008,FLOOD,Flood,DR,Statewide,IA,7,1953-06-11,1953-06-11,0,0,1,1,1,f100b5c1-0f3f-4c3d-ad3c-50040e98ed8d
67353,7,53009,TORNADO,Tornado,DR,Statewide,MA,1,1953-06-11,1953-06-11,0,0,1,1,1,fca83efc-524a-45a0-ac98-2d43b954a5ea
67354,2,53003,TORNADO & HEAVY RAINFALL,Tornado,DR,Statewide,TX,6,1953-05-15,1953-05-15,0,0,1,1,1,ff821327-6b90-4246-b19f-fff8c4b288a8


#### Sampling Disaster Types

In [4]:
# Sampling of unique FEMA Disaster Types
disaster_types = fema_decl_clean['incidentType'].unique()
pprint(disaster_types)

array(['Fire', 'Severe Storm', 'Straight-Line Winds', 'Flood',
       'Winter Storm', 'Hurricane', 'Tornado', 'Mud/Landslide',
       'Tropical Storm', 'Snowstorm', 'Earthquake', 'Coastal Storm',
       'Other', 'Severe Ice Storm', 'Biological', 'Dam/Levee Break',
       'Typhoon', 'Volcanic Eruption', 'Freezing', 'Toxic Substances',
       'Chemical', 'Terrorist', 'Drought', 'Human Cause',
       'Fishing Losses', 'Tsunami'], dtype=object)


#### Merging Natural Disasters Key

In [5]:
# Filter for just natural disaster types
disaster_natural = pd.DataFrame(
    {
        'incidentType': [
            'Fire', 'Severe Storm', 'Straight-Line Winds', 
            'Flood', 'Winter Storm', 'Hurricane', 'Tornado',
            'Tropical Storm', 'Snowstorm', 'Coastal Storm', 
            'Severe Ice Storm', 'Typhoon', 'Freezing', 'Drought', 
            'Tsunami'
        ]
    }
)

# Merging with natural disaster key for only designated types
fema_disasters = pd.merge(fema_decl_clean, disaster_natural, how = 'right', on = 'incidentType')
fema_disasters.to_csv('Assets/CSVs/femaDisasters_dirty.csv', index = False)

display(fema_disasters)

,disasterNumber,declarationRequestNumber,declarationTitle,incidentType,declarationType,designatedArea,state,region,declarationDate,incidentBeginDate,tribalRequest,ihProgramDeclared,iaProgramDeclared,paProgramDeclared,hmProgramDeclared,id
0,5530,24123,GOLD RANCH FIRE,Fire,FM,Washoe (County),NV,9,2024-08-12,2024-08-11,0,0,0,1,1,f15a7a79-f1c3-41bb-8a5c-c05fbae34423
1,5529,24122,LEE FALLS FIRE,Fire,FM,Washington (County),OR,10,2024-08-09,2024-08-08,0,0,0,1,1,09e3f81a-5e16-4b72-b317-1c64e0cfa59c
2,5528,24116,ELK LANE FIRE,Fire,FM,Jefferson (County),OR,10,2024-08-06,2024-08-04,0,0,0,1,1,59983f89-30bf-4888-b21b-62e8d57d9aac
3,5527,24111,MILE MARKER 132 FIRE,Fire,FM,Deschutes (County),OR,10,2024-08-02,2024-08-02,0,0,0,1,1,8d13ecf0-bc2f-496b-8c9f-b2e73da832a0
4,5522,24102,BOREL FIRE,Fire,FM,Kern (County),CA,9,2024-07-27,2024-07-25,0,0,0,1,1,9b73e19b-d326-4992-8da1-7e658d97607c
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58774,1967,11042,TSUNAMI WAVES,Tsunami,DR,Honolulu (County),HI,9,2011-04-08,2011-03-11,0,0,0,1,1,9ce5ad8d-f0e1-4d12-9907-a704fe2f4511
58775,1967,11042,TSUNAMI WAVES,Tsunami,DR,Maui (County),HI,9,2011-04-08,2011-03-11,0,0,0,1,1,d64d8673-10bc-4799-809b-70317d714d68
58776,1964,11029,TSUNAMI WAVE SURGE,Tsunami,DR,Coos (County),OR,10,2011-03-25,2011-03-11,0,0,0,1,1,35f729e0-73ff-42aa-9b80-8bcc901be612
58777,1964,11029,TSUNAMI WAVE SURGE,Tsunami,DR,Curry (County),OR,10,2011-03-25,2011-03-11,0,0,0,1,1,89c45999-d563-423f-a2dd-e69b96103b55


### Exploration

In [6]:
# Counts of declaration types 
# DR = Major Disasters, EM = Emergency Declaratation, FM = Fire Management)
# For more info see README (in progress)
fema_disasters['declarationType'].value_counts()

declarationType
DR    40878
EM    15907
FM     1994
Name: count, dtype: int64

In [7]:
# Counts of natural disaster types
fema_disasters['incidentType'].value_counts()

incidentType
Severe Storm           18399
Hurricane              13721
Flood                  11093
Fire                    3759
Snowstorm               3707
Severe Ice Storm        2942
Tornado                 1623
Drought                 1292
Tropical Storm          1047
Coastal Storm            637
Freezing                 301
Typhoon                  130
Winter Storm             117
Tsunami                    9
Straight-Line Winds        2
Name: count, dtype: int64

In [8]:
# Validating no NaNs
fema_disasters.count()

disasterNumber              58779
declarationRequestNumber    58779
declarationTitle            58779
incidentType                58779
declarationType             58779
designatedArea              58779
state                       58779
region                      58779
declarationDate             58779
incidentBeginDate           58779
tribalRequest               58779
ihProgramDeclared           58779
iaProgramDeclared           58779
paProgramDeclared           58779
hmProgramDeclared           58779
id                          58779
dtype: int64

In [9]:
# Filter: Natural disasters per State
fema_disasters['state'].value_counts()

state
TX    4779
FL    2635
KY    2554
MO    2534
LA    2398
GA    2333
OK    2305
VA    2298
NC    2109
PR    1835
MS    1777
IA    1719
KS    1607
AL    1603
TN    1532
CA    1525
AR    1472
MN    1406
NE    1336
ND    1280
IN    1279
NY    1272
SD    1181
WV    1127
PA    1118
IL    1109
OH    1100
SC    1039
WA     827
WI     725
MI     595
NJ     556
CO     531
OR     530
MT     507
ME     426
MD     399
NM     380
VT     377
MA     365
NH     300
AZ     271
ID     265
CT     245
NV     197
AK     187
UT     181
RI     106
WY      86
HI      85
VI      79
MP      67
AS      61
MH      53
DE      47
FM      31
GU      19
DC      18
PW       1
Name: count, dtype: int64

In [10]:
# Filter: Natural disasters per hit area
fema_disasters['designatedArea'].value_counts()

designatedArea
Washington (County)                           507
Jefferson (County)                            460
Jackson (County)                              450
Statewide                                     446
Franklin (County)                             439
                                             ... 
Alabama and Coushatta Indian Reservation        1
Ysleta del Sur Pueblo (Indian Reservation)      1
Nondalton (ANV/ANVSA)                           1
Kusilvak Census Area                            1
Namorik (County-equivalent)                     1
Name: count, Length: 2187, dtype: int64

## *Phase 2*

### Sampling DataFrame - Visual

In [11]:
fema_disasters.head(10)

,disasterNumber,declarationRequestNumber,declarationTitle,incidentType,declarationType,designatedArea,state,region,declarationDate,incidentBeginDate,tribalRequest,ihProgramDeclared,iaProgramDeclared,paProgramDeclared,hmProgramDeclared,id
0,5530,24123,GOLD RANCH FIRE,Fire,FM,Washoe (County),NV,9,2024-08-12,2024-08-11,0,0,0,1,1,f15a7a79-f1c3-41bb-8a5c-c05fbae34423
1,5529,24122,LEE FALLS FIRE,Fire,FM,Washington (County),OR,10,2024-08-09,2024-08-08,0,0,0,1,1,09e3f81a-5e16-4b72-b317-1c64e0cfa59c
2,5528,24116,ELK LANE FIRE,Fire,FM,Jefferson (County),OR,10,2024-08-06,2024-08-04,0,0,0,1,1,59983f89-30bf-4888-b21b-62e8d57d9aac
3,5527,24111,MILE MARKER 132 FIRE,Fire,FM,Deschutes (County),OR,10,2024-08-02,2024-08-02,0,0,0,1,1,8d13ecf0-bc2f-496b-8c9f-b2e73da832a0
4,5522,24102,BOREL FIRE,Fire,FM,Kern (County),CA,9,2024-07-27,2024-07-25,0,0,0,1,1,9b73e19b-d326-4992-8da1-7e658d97607c
5,5521,24098,RETREAT FIRE,Fire,FM,Yakima (County),WA,10,2024-07-26,2024-07-23,0,0,0,1,1,bc02a243-2458-4af3-84e3-b9928bd1d41d
6,5520,24099,GWEN FIRE,Fire,FM,Nez Perce Indian Reservation,ID,10,2024-07-26,2024-07-25,0,0,0,1,1,85cccdec-5e22-436f-a3e7-acd7340172cf
7,5520,24099,GWEN FIRE,Fire,FM,Latah (County),ID,10,2024-07-26,2024-07-25,0,0,0,1,1,d0e4282c-5d39-4af2-814c-2192d748f16c
8,5520,24099,GWEN FIRE,Fire,FM,Nez Perce (County),ID,10,2024-07-26,2024-07-25,0,0,0,1,1,b058f371-8e3e-49dc-bb08-aac48229a85f
9,5519,24097,PARK FIRE,Fire,FM,Butte (County),CA,9,2024-07-25,2024-07-24,0,0,0,1,1,00045754-7992-448a-a8ae-181fb748a576


### Advanced Cleanup - RegEx

#### Area/AreaType Split

In [12]:
# Splitting up area from the areaType in designatedArea
geocode_df = fema_disasters[['designatedArea', 'state', 'id']].copy()
geocode_df[['area', 'areaType']] = geocode_df['designatedArea'].str.split('(', n = 1, expand = True)
geocode_df = geocode_df.drop(axis = 0, columns = 'designatedArea')
areas = geocode_df.pop('area')
areaTypes = geocode_df.pop('areaType')
areaTypes = '(' + areaTypes
geocode_df.insert(0, 'area', areas)
geocode_df.insert(1, 'areaType', areaTypes)

#### AreaType Cleaning

##### Removing Symbols/Conjunctions

In [13]:
# Regex for cleaning up text in areaType
geocode_df['areaType'] = geocode_df['areaType'].str.extract(r'\(([^)]+)\)')
geocode_df['areaType'] = geocode_df['areaType'].str.replace(r'\b(of|Also|and)\b', '', regex = True).str.strip()
display(geocode_df)

,area,areaType,state,id
0,Washoe,County,NV,f15a7a79-f1c3-41bb-8a5c-c05fbae34423
1,Washington,County,OR,09e3f81a-5e16-4b72-b317-1c64e0cfa59c
2,Jefferson,County,OR,59983f89-30bf-4888-b21b-62e8d57d9aac
3,Deschutes,County,OR,8d13ecf0-bc2f-496b-8c9f-b2e73da832a0
4,Kern,County,CA,9b73e19b-d326-4992-8da1-7e658d97607c
...,...,...,...,...
58774,Honolulu,County,HI,9ce5ad8d-f0e1-4d12-9907-a704fe2f4511
58775,Maui,County,HI,d64d8673-10bc-4799-809b-70317d714d68
58776,Coos,County,OR,35f729e0-73ff-42aa-9b80-8bcc901be612
58777,Curry,County,OR,89c45999-d563-423f-a2dd-e69b96103b55


##### Checking areaTypes Found Multi-State Inputs

In [14]:
# Sampling areaTypes after regex based edit
geocode_df['areaType'].unique()

array(['County', nan, 'Parish', 'Borough', 'Census Area',
       'Indian Reservation', 'Municipio', 'ANV/ANVSA', 'NM',
       'County-equivalent', 'OTSA', 'NM  UT', 'Sisseton', 'Reservation',
       'District', 'Island', 'AZ  UT', 'Township',
       'Unorganized Territory', 'Police Jury Ward', 'Nett Lake', 'TDSA',
       'KS', 'Municipality', 'CCD', 'State', 'Census Subarea'],
      dtype=object)

##### Duplicate Rows by Multi-State

In [15]:
# Setting up the Data Frame containing Area Types with 1
# or more states instead of classic 'County' or 'Reservation'
# --> Result is duplicated incidents but for the additional states

# Multi-state regex location in 'areaType' column
multiState_regex = r'\b[A-Z]{2}(?:\s[A-Z]{2})*\b'
states_to_split = geocode_df['areaType'].str.contains(multiState_regex, na = False)
new_rows = []

# Iterate through just the Area Types that need to be split and duplicated
for index, row in geocode_df[states_to_split].iterrows():
    states = row['areaType'].split()
    for state in states:
        new_row = row.copy()
        new_row['state'] = state
        new_rows.append(new_row)

geocode_df = pd.concat(     # Concatenate duplicated rows for multiple states to the main dataframe
    [geocode_df, pd.DataFrame(new_rows)], ignore_index = True
    ).reset_index(drop = True).copy()

##### Combining Native American Area Types

In [16]:
# Set Non-Capture Regex and location rows that can have areaType set to 'Reservation'
reservation_regex = r'(?:Indian)|(?:Reservation)|(?:Tribe)|(?:Trust Lands)|(?:TDSA)'
reservation_loc = geocode_df['area'].str.contains(reservation_regex, case = False, regex = True)
geocode_df.loc[reservation_loc, 'areaType'] = 'Native Land'

##### Fixing Specific Located areaType  

Definitely too nitty-gritty

In [17]:
# Using geocode_df['areaType'].unique() location type 'City  Borough'
# Only 1 entry of Wranell, AK (Alaska) -> Correct to 'Borough'
geocode_df.loc[geocode_df['areaType'] == 'City  Borough', 'areaType'] = 'Borough'

#### Area Cleaning

##### Mapping 'Statewide' to State Name

In [18]:
# Creating Dictionary of State Abbr. --> State Full Name for mapping to DataFrame
# Mapping resolves issue of 'Statewide' Listed Area into a the full name of the State
# areaType is then switched to 'State'
state_dict = {
    'AK': 'Alaska', 'AL': 'Alabama', 'AR': 'Arkansas', 'AS': 'American Samoa', 'AZ': 'Arizona', 
    'CA': 'California', 'CO': 'Colorado', 'CT': 'Connecticut', 'DC': 'District of Columbia', 
    'DE': 'Delaware', 'FL': 'Florida', 'GA': 'Georgia', 'GU': 'Guam', 'HI': 'Hawaii', 'IA': 'Iowa', 
    'ID': 'Idaho', 'IL': 'Illinois', 'IN': 'Indiana', 'KS': 'Kansas', 'KY': 'Kentucky', 'LA': 'Louisiana', 
    'MA': 'Massachusetts', 'MD': 'Maryland', 'ME': 'Maine', 'MI': 'Michigan', 'MN': 'Minnesota', 
    'MO': 'Missouri', 'MP': 'Northern Mariana Islands', 'MS': 'Mississippi', 'MT': 'Montana', 
    'NC': 'North Carolina', 'ND': 'North Dakota', 'NE': 'Nebraska', 'NH': 'New Hampshire', 'NJ': 'New Jersey', 
    'NM': 'New Mexico', 'NV': 'Nevada', 'NY': 'New York', 'OH': 'Ohio', 'OK': 'Oklahoma', 'OR': 'Oregon', 
    'PA': 'Pennsylvania', 'PR': 'Puerto Rico', 'RI': 'Rhode Island', 'SC': 'South Carolina', 
    'SD': 'South Dakota', 'TN': 'Tennessee', 'TX': 'Texas', 'UT': 'Utah', 'VA': 'Virginia', 
    'VI': 'U.S. Virgin Islands', 'VT': 'Vermont', 'WA': 'Washington', 'WI': 'Wisconsin', 
    'WV': 'West Virginia', 'WY': 'Wyoming'
}

# Found under Null types (NaN)
# Location Statewide listed areas, copying abbreviations from `state` column
# to the `area` column, extending them to full length names and then setting 
# the `areaType` to State
statewide_locs = (geocode_df['area'] == 'Statewide')    # Bool location
geocode_df.loc[statewide_locs, 'areaType'] = 'State'    # Set type to State
geocode_df.loc[statewide_locs, 'area'] = geocode_df.loc[statewide_locs, 'state']    # Copy State value to area
geocode_df['area'] = geocode_df['area'].replace(state_dict)   # Transform abbr states to Full name

##### Regional Educational Attendance Areas

In [19]:
# Regional Educational Attendance Area `area` with Null `areaType` (it's the same with or without null spec)
reaa_regex = r'Regional Educational.*$'
reaa_loc = geocode_df['area'].str.contains(reaa_regex, case = False, regex = True)
geocode_df.loc[reaa_loc, 'areaType'] = 'County'
geocode_df.loc[reaa_loc, 'area'] = geocode_df.loc[reaa_loc, 'area'].str.replace(reaa_regex, '', regex = True)

### Binning AreaTypes - Granularization

In [20]:
# Organizing `areaType` into bins for granularity
areaType_bins = {
    'State': ['State', 'Territory', 'Unorganized Territory', 'Island'],
    'County': ['County', 'County-equivalent', 'Parish', 'Borough', 'Census Area'],
    'City': ['City', 'Township', 'Municipality', 'Municipio', 'Census Subarea', 'CCD'],
    'Native Land': ['Native Land', 'Reservation', 'Indian Reservation', 'ANV/ANVSA', 'OTSA', 'TDSA', 'Native Regional Corporation', 'Nett Lake'],
    'Other': ['Other', 'District', 'Police Jury Ward', 'Sisseton']
}

# Converting bins into reverse-lookup for mapping
# (Treats dictionary created with list-comprehension)
areaType_map = {
    old_type: new_type  # Specifying key:value pair setup
    for new_type, old_types in areaType_bins.items()    # Outer-iteration of type_key -> [type_val1, type_val2, ...]
    for old_type in old_types   # Inner-iteration of type in [type1, type2, type3, ...]
}

# Set mapping to DataFrame for corrections
geocode_df['areaType'] = geocode_df['areaType'].map(areaType_map, na_action = 'ignore')

### Drop Nulls & Strip Whitespace
Final cleaning before Geocoding

In [21]:
# Drop null rows after filtering for null corrections
# Drop columns used for accurate location derivation
geocode_df = geocode_df.dropna()
fema_disasters = fema_disasters.drop(axis = 0, columns = ['designatedArea', 'state'])

# Finally correcting whitespace issues for file saving
geocode_df['area'] = geocode_df['area'].str.strip().str.title()
geocode_df['state'] = geocode_df['state'].str.strip()

## *Phase 3*

### **Client to API Ops**
---

#### Base Geoapify API Call Function

In [22]:
# Function definition for Geoapify API call
# Restricted by county type and manually created rectangle via coordinate pairs
# API Key is a variable argument given by outer functions calling geoapify_fetch()


# Country Codes: 
    # as: American Samoa, fm: Micronesia, gu: Guam, 
    # mh: Marshall Islands, mp: Norhtern Mariana Islands, 
    # pr: Puerto Rico, vi: Virgin Islands, 
    # um: US Minor Outlying Islands, us: USA


def geoapify_fetch(area, state, key):
    # Geoapify base API URL
    geoapify_base = 'https://api.geoapify.com/v1/geocode/search?'

    # Define limiting rectangle for filter use
    box = {
        'lat_min': 16,
        'lat_max': 70,
        'lng_min': -168,
        'lng_max': -53
    }

    # Paremeter setting
    countryCodes = 'us,um,pr,vi'
    rectangle = f"{box['lng_min']},{box['lat_min']},{box['lng_max']},{box['lat_max']}"
    geoapify_params = {
        'filter': f'rect:{rectangle}|countrycode:{countryCodes}',   # Filtering by rectangle and country code for good data
        'text': f'{area}, {state}',                                 # Freeform text query
        'apiKey': key                                               # Varying API Key from threading by multiple keys
    }

    try:
        # Request, conversion, and parsing for function returns - Returns Nulls for no response or error
        geo_response = requests.get(geoapify_base, geoapify_params).json()
        if geo_response['features']:
            lng, lat = geo_response['features'][0]['geometry']['coordinates']
            lat = float(lat)
            lng = float(lng)
            print(f'Fetching {area}, {state}: {lat}, {lng}')        # Log printing - Success!
            return (area, state), (lat, lng)                        # Returning 2x2 Tuples
        else:
            print(f'No result for: {area}, {state}')                # Log printing - No Response
            return (area, state), None                              # Only returning initial tuple key given
    except Exception as e:
        print(f'Error return for {area}, {state}: {e}')             # Log printing - Error Handling
        return (area, state), None                                  # Only returning initial tuple key given
    finally:
        sleep(0.2)                                                  # Respecting rate limit per key no matter what

#### Safety Wrapper for Geoapify Fetch

In [23]:
# When integrating multi-threading there were multiple issues from input formatting
# --> Safety wrapper created to morph data for API calling
def safe_geoapify_fetch(row_iter, key):
    try:
        _, row = row_iter
        area = row['area']
        state = row['state']
        return (area, state), geoapify_fetch(area, state, key)
    except Exception as e:
        print(f'Error fetching data for {row}: {e}')

#### Call Execution Using Multi-Thread

In [24]:
# Creating Path variable for easy CSV grabbing
# To run API Call delete existing CSV file or change path name
geocode_file = Path('Assets/CSVs/location_geocode.csv')

# Checking for Path non-existence -> Run main API call
if not Path.exists(geocode_file):
    # 9 API Keys in 'api_keys.py' poured into a list
    keys = [
        api_keys.geoapify_NEEL_1,
        api_keys.geoapify_NEEL_2,
        api_keys.geoapify_NEEL_3,
        api_keys.geoapify_NEEL_4,
        api_keys.geoapify_KENDALL,
        api_keys.geoapify_ALEX_1,
        api_keys.geoapify_ALEX_2,
        api_keys.geoapify_AVENIKA,
        api_keys.geoapify_CLAUDIA
    ]
    
    # Creating DataFrame of unique locations by area, state
    # Sorted values by state, area for logging progress visibility
    unique_loc = geocode_df[['area', 'state']
                            ].drop_duplicates(
                            ).sort_values(['state', 'area']
                            ).reset_index(drop = True)


    # <-- API CALL WITH INTEGRATED MULTI-THREADING
    api_calls = len(unique_loc)
    print(
        'BEGIN GEOAPIFY API FETCH:\n' +
        f'Locating {api_calls} unique places...\n' + 
        '*************************************\n'
    )
    # Creating blank dictionary for DF mapping post API call
    loc_dict = {}

    # Execute Geoapify Fetch function with a thread for each key
    # Max workers are determined by the # of API Keys
    # Future objects are returned into a list 
    with ThreadPoolExecutor(max_workers = len(keys)) as executor:
        results = list(
            executor.map(                                                       # Mapping each executor thread to API Call safety wrapper
                lambda row_key: safe_geoapify_fetch(row_key[0], row_key[1]),    # Give each Safety wrapper the zipped arguments
                zip(unique_loc.iterrows(), cycle(keys))                         # Create args of row iterable and inf cycle of keys
            )
        )

    # Fill loc_dict with results from threads for mapping
    # Unpacking by results -> (location, coords) -> coords
    for result in results:
        if result:              # Checking for result existence from caught API errors
            location, coords = result
            if coords:          # Checking for coord existence from caught response issues
                loc_dict[location] = coords

    print(
    '\n*************************************\n' + 
    'END OF GEOAPIFY API FETCH\n'
    )
    # END OF API CALL THREADING /-->


    # Mapping latitudes and longitudes by loc_dict
    # Getting value from key and retriving only the 2nd entry
    geocode_df['coords'] = geocode_df.apply(
        lambda row: loc_dict.get((row['area'], row['state']), ((None, None), (None, None)))[1],
        axis = 1
    )
    # Ensuring grouping is of tuples and no longer tuple of tuples
    geocode_df['coords'] = geocode_df['coords'].apply(
        lambda x: x if isinstance(x, tuple) else (None, None)
    )

    # Split coordinates into separate columns via converting to list then DataFrame
    geocode_df[['latitude', 'longitude']] = pd.DataFrame(geocode_df['coords'].tolist(), index = geocode_df.index)

    # Dropping coordinate column after pair splitting
    geocode_df = geocode_df.drop(axis = 0, columns = 'coords')

    # Ensuring columns are the correct type before CSV saving
    geocode_df['latitude'] = geocode_df['latitude'].astype(float)
    geocode_df['longitude'] = geocode_df['longitude'].astype(float)

    # Saving to CSV to prevent unnecessary executions
    geocode_df.to_csv(geocode_file, index = False)
else:
    # Result of a pre-existing CSV File under the Path type variable declared pre conditional block
    print(
        '*****************\n' +
        'API CALL SKIPPED!\n' +
        '*****************\n' +
        'File created from combined API results and DataFrame already exists.\n' +
        f'In order to run API call, please delete {geocode_file.name} and try again.'
    )
    # Reading in pre-existing CSV to maintain jumping-off point after API Call
    geocode_df = pd.read_csv(geocode_file)

BEGIN GEOAPIFY API FETCH:
Locating 3427 unique places...
*************************************

Fetching Aleutians East, AK: 55.0512443, -162.8916892
Fetching Bristol Bay, AK: 58.7370341, -156.8753867
Fetching Anchorage, AK: 61.2163129, -149.894852
Fetching Aleutian Islands, AK: 54.720935049999994, -164.04080923135757
Fetching Bethel, AK: 61.3204199, -159.3709719
Fetching Bering Strait, AK: 64.502012, -165.407103
Fetching Alaska Gateway, AK: 61.562911, -149.256134
Fetching Alaska, AK: 61.1744097, -149.9354120350149
Fetching Chignik Lake, AK: 56.2530286, -158.763948
Fetching Aleutians West, AK: 55.430915, -161.705244
Fetching Chugach, AK: 61.15374635, -149.84777862232738
Fetching Chignik Lagoon, AK: 63.6833333, -149.8569444
Fetching Copper River, AK: 62.5688654, -143.6982537
Fetching Dillingham, AK: 59.9011107, -158.2016852
Fetching Dot Lake, AK: 63.6617238, -144.0636789
Fetching Fairbanks North Star, AK: 64.8649039, -146.7751619
Fetching Denali, AK: 63.878678, -149.650166
Fetching June

### Geocoded Data ETL

#### Sampling

In [25]:
# Displaying newly retrieved data
geocode_df.head()

,area,areaType,state,id,latitude,longitude
0,Washoe,County,NV,f15a7a79-f1c3-41bb-8a5c-c05fbae34423,40.584905,-119.613161
1,Washington,County,OR,09e3f81a-5e16-4b72-b317-1c64e0cfa59c,45.560188,-123.058791
2,Jefferson,County,OR,59983f89-30bf-4888-b21b-62e8d57d9aac,44.722434,-123.007389
3,Deschutes,County,OR,8d13ecf0-bc2f-496b-8c9f-b2e73da832a0,44.156923,-121.258700
4,Kern,County,CA,9b73e19b-d326-4992-8da1-7e658d97607c,35.314570,-118.753822


#### Merge | Clean | Save

In [26]:
# Merge with DataFrame -> Work from fema_disasters now
# Drop id's and Nulls of any kind, as result will be rows with coordinates to them
fema_disasters = fema_disasters.merge(geocode_df, how = 'left', on = 'id').copy()
fema_disasters = fema_disasters[['disasterNumber', 'declarationRequestNumber', 'declarationTitle', 
                                'area', 'areaType', 'state', 'incidentType', 'declarationType', 
                                'declarationDate', 'incidentBeginDate', 'id', 'latitude', 'longitude', 
                                'tribalRequest', 'ihProgramDeclared', 'iaProgramDeclared',
                                'paProgramDeclared', 'hmProgramDeclared']]
fema_disasters = fema_disasters.drop(axis = 0, columns = 'id')
fema_disasters = fema_disasters.dropna(how = 'any')
fema_disasters.head(10)

fema_disasters.to_csv('Assets/CSVs/femaDisasters_clean.csv', index = False)

### Summary by Disaster Numbers

Please note the unique structure of the disaster sequencing (due to a numbering system that originated in the 1950's-1970's):

0001-1999 Major Disaster Declaration

2000-2999 Fire Management

3000-3999 Emergency Declaration (Special Emergency)

4000-4999 Major Disaster Declaration

5000-5999 Fire Management

In [27]:
# Grouping to find summaries by disaster number
disaster_byNumber = fema_disasters.groupby('disasterNumber')

# Collecting non-unique titles under disaster number
titles_byDisaster = disaster_byNumber['declarationTitle'].first()
incidents_byDisaster = disaster_byNumber['incidentType'].first()

# Counting declarations per disaster and retrieving non-unique declaration number
declRequest_byDisaster_NUM = disaster_byNumber['declarationRequestNumber'].count()
declRequest_byDisaster = disaster_byNumber['declarationRequestNumber'].min().astype(str)

# Collecting first and last declaration dates
declDate_byDisaster_MIN = disaster_byNumber['declarationDate'].min()
declDate_byDisaster_MAX = disaster_byNumber['declarationDate'].max()

# Collecting first and last incident beginning dates
incDate_byDisaster_MIN = disaster_byNumber['incidentBeginDate'].min()
incDate_byDisaster_MAX = disaster_byNumber['incidentBeginDate'].max()

# Collecting average latitude and longitude per disaster
lat_byDisaster = disaster_byNumber['latitude'].mean()
lng_byDisaster = disaster_byNumber['longitude'].mean()

# Creating table of aggregated and collected values
disaster_summary = pd.DataFrame(
    {
        'Title': titles_byDisaster,
        'Number of Requests': declRequest_byDisaster_NUM,
        'Declaration Request Number': declRequest_byDisaster,
        'Incident Type': incidents_byDisaster,
        'Incident Begin Date (First)': incDate_byDisaster_MIN,
        'Incident Begin Date (Last)': incDate_byDisaster_MAX,
        'Declaration Date (First)': declDate_byDisaster_MIN,
        'Declaration Date (Last)': declDate_byDisaster_MAX,
        'Latitude': lat_byDisaster,
        'Longitude': lng_byDisaster
    }
)
# Reinding by disaster number
disaster_summary.index.names = ['Disaster Number']

# Saving to CSV
disaster_summary.to_csv('Assets/CSVs/disasters_summary.csv')

In [28]:
disaster_summary.head(10)

,Title,Number of Requests,Declaration Request Number,Incident Type,Incident Begin Date (First),Incident Begin Date (Last),Declaration Date (First),Declaration Date (Last),Latitude,Longitude
Disaster Number,,,,,,,,,,
1,TORNADO,1,53013,Tornado,1953-05-02,1953-05-02,1953-05-02,1953-05-02,30.876607,-84.200199
2,TORNADO & HEAVY RAINFALL,1,53003,Tornado,1953-05-15,1953-05-15,1953-05-15,1953-05-15,29.396013,-94.917548
3,FLOOD,1,53005,Flood,1953-05-29,1953-05-29,1953-05-29,1953-05-29,32.787346,-91.904878
4,TORNADO,1,53004,Tornado,1953-06-02,1953-06-02,1953-06-02,1953-06-02,42.233092,-84.327177
5,FLOODS,1,53006,Flood,1953-06-06,1953-06-06,1953-06-06,1953-06-06,46.540855,-111.946345
6,TORNADO,1,53007,Tornado,1953-06-09,1953-06-09,1953-06-09,1953-06-09,42.233092,-84.327177
7,TORNADO,1,53009,Tornado,1953-06-11,1953-06-11,1953-06-11,1953-06-11,42.358752,-71.531484
8,FLOOD,1,53008,Flood,1953-06-11,1953-06-11,1953-06-11,1953-06-11,41.683983,-92.061227
9,FLOOD,1,53010,Flood,1953-06-19,1953-06-19,1953-06-19,1953-06-19,29.396013,-94.917548
